In [1]:
import numpy as np
import gym
import gym_custom_envs
import time
import random
import scipy.linalg as alg
# import control

In [2]:
from IPython.display import clear_output
env = gym.make("Quadrotor2D-v0")
# uncomment for testing environment below
# for i in range(5):
#     s = env.reset()
    
#     done = False
#     while not done:
#         a = np.array(np.random.uniform(low=-2.0, high=2.0, size=(2,)))
# #         a = np.zeros(2)
#         print(round(s[0], 3), round(s[1], 3), round(np.rad2deg(s[2]), 3))
#         ns, r, done, _ = env.step(a)
#         s = ns
#         env.render()
#         time.sleep(0.08)
#         clear_output(wait=True)

In [3]:
m = env.m
I = env.I
g = env.gravity
r = env.l/2

# M(q) & B(q) since C(q, q_dot) and Tau(q) is zero
M = np.matrix([
    [m, 0, 0],
    [0, m, 0],
    [0, 0, I],
])

b = np.matrix([
    np.zeros(2),
    [1, 1],
    [r, -r]
])

iM = np.linalg.inv(M)
print('M(q) = ')
print(M)

print('b(q) = ')
print(b)

print('iM(q) = ')
print(iM)

M(q) = 
[[1.         0.         0.        ]
 [0.         1.         0.        ]
 [0.         0.         0.02083333]]
b(q) = 
[[ 0.    0.  ]
 [ 1.    1.  ]
 [ 0.25 -0.25]]
iM(q) = 
[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0. 48.]]


In [4]:
def lqr(A, B, Q, R):
#     db = np.matrix([
#         [0 , 0, -u[0]-u[1]],
#         np.zeros(3),
#         np.zeros(3)
#         ])
#     x = np.array([0, -g/X[1], 0]).reshape(3, 1)
#     print(x)
#     db = np.concatenate((np.zeros((3, 1)),
#                         x,
#                         np.zeros((3, 1))), axis=1)
#     print(db)
#     A = np.concatenate((
#         np.concatenate((np.zeros((3, 3)), np.eye(3)), axis=1),
#         np.concatenate((np.dot(iM, db), np.zeros((3, 3))), axis=1)
#     ), axis=0)
#     print(A)
#     B = np.concatenate(
#         (np.zeros((3, 3))
#         , np.matrix([
#         [0, -X[2], -X[2]],
#         [0, 1, 1],
#         [0, r, r]
#     ])), axis=0)
    
    S = np.matrix(alg.solve_continuous_are(A, B, Q, R))
    
    K = np.matrix(alg.inv(R)*B.T*S)
    
#     K, S, E = control.lqr(A, B, Q, R)
    
    return K, S

In [5]:
db = np.matrix([
    [0 , 0, -g],
    np.zeros(3),
    np.zeros(3)
    ])

A = np.concatenate((
    np.concatenate((np.zeros((3, 3)), np.eye(3)), axis=1),
    np.concatenate((np.dot(iM, db), np.zeros((3, 3))), axis=1)
), axis=0)

B = np.concatenate((
    np.zeros((3, 2)),
    np.dot(iM, b)
), axis=0)

# A = np.concatenate((
#     np.concatenate((np.zeros((3, 3)), np.eye(3)), axis=1),
#     np.zeros((3, 6))
# ), axis=0)

# B = np.concatenate((
#     np.zeros((4, 2)),
#     np.matrix([
#         [1/m, 1/m],
#         [r/I, r/I]
#     ])
# ), axis=0)

# Q = np.eye(6)

# R = np.eye(3)
Q = np.diag([1, 1, 1, 1, 1, 1])
R = np.array([[0.1, 0], [0, 0.1]])

K, S = lqr(A, B, Q, R)

print('A = ')
print(A)
print('B = ')
print(B)

print('Q = ')
print(Q)

print('R = ')
print(R)

print('K = ')
print(K)

A = 
[[  0.   0.   0.   1.   0.   0.]
 [  0.   0.   0.   0.   1.   0.]
 [  0.   0.   0.   0.   0.   1.]
 [  0.   0. -10.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.]]
B = 
[[  0.   0.]
 [  0.   0.]
 [  0.   0.]
 [  0.   0.]
 [  1.   1.]
 [ 12. -12.]]
Q = 
[[1 0 0 0 0 0]
 [0 1 0 0 0 0]
 [0 0 1 0 0 0]
 [0 0 0 1 0 0]
 [0 0 0 0 1 0]
 [0 0 0 0 0 1]]
R = 
[[0.1 0. ]
 [0.  0.1]]
K = 
[[ -2.23606798   2.23606798  12.82917841  -3.27679462   2.68999405
    2.46355398]
 [  2.23606798   2.23606798 -12.82917841   3.27679462   2.68999405
   -2.46355398]]


In [ ]:
for i in range(50):
    s = env.reset().reshape(1, 6)
    
    done = False
    
    while not done:
        K, S = lqr(A, B, Q, R)
        
        a = np.dot(K, s.T)
#         a = np.clip(a, -env.thrust_max, env.thrust_max)
        print('Theta = ', round(np.rad2deg(s[0, 2]), 3))
        print('u = ', a.T)
        print("Reaction = ", round((a[0] + a[1])[0, 0] + m*g, 3))
        print("Cost = ", round(np.dot(s, np.dot(S, s.T))[0, 0], 3))
#         a = [0, 0]
#         a = [g/2, g/2]
        ns, r, done, _ = env.step([a[0], a[1]])
        s = ns.reshape(1, 6)
        env.render()
        clear_output(wait=True)
        time.sleep(0.08)